# Batch correction and integrated analysis of ependymoma samples
Regrettably, seems gojo et al aggregated all of their sequencing by sample, so impossible now to
batch correct on sequencing type (scSmart-seq2, snSmart-seq2, 10X Genomics) without reanalyzing from scratch.


In [ ]:
# Load all required libraries
Sys.setenv(LANGUAGE = "en") # set language to "ja" if you prefer
library(Seurat)
library(tidyverse)
library(harmony)
library(ggplot2)
library(future)
library(ggalluvial)
library(mclust)
library(patchwork)

In [ ]:
# Set parallel execution settings
future::plan("multisession", workers = as.integer(availableCores()/2), gc = TRUE)
options(future.globals.maxSize = 1024*8*1024^2) # Set max variable size to 8Gb

In [ ]:
# Load data
DATA_DIR = file.path('data','external','gojo_et_al') # change this if you put your data somewhere other than ./data/external/gojo_et_al/
import_gojo_data <- function(counts_file,metadata_file,path){
    counts_path = file.path(DATA_DIR,counts_file)
    metadata_path = file.path(DATA_DIR,metadata_file)
    counts = read.table(counts_path)
    meta = read.table(metadata_path)    
    obj <- CreateSeuratObject(counts = counts , meta.data = meta) %>% suppressWarnings
    return(obj)
}
clean_pdx <- function(pdx){
    # clean the pdx metadata.
    # Delete column V1 (duplicate of row indices)
    # rename column V2
    names(pdx[[]])[names(pdx[[]]) == 'V2'] <- 'sample'
    pdx@meta.data[c('V1','V2')] <- NULL
    pdx$annotation <- pdx$sample
    return(pdx)
}

In [ ]:
# Load data for ST samples

#pf = import_gojo_data('PF_EPN_counts_200519lj.txt','PF_EPN_metadata_200519lj.txt',DATA_DIR)
#sp = import_gojo_data('SP_EPN_counts_200519lj.txt','SP_EPN_metadata_200519lj.txt',DATA_DIR)
st = import_gojo_data('ST_EPN_counts_200519lj.txt','ST_EPN_metadata_200519lj.txt',DATA_DIR)
pdx = import_gojo_data('PDX_counts.txt','PDX_metadata.txt',DATA_DIR) %>% clean_pdx
#pairs = import_gojo_data('Matched_pair_counts_200519lj.txt','Matched_pair_metadata_200519lj.txt',DATA_DIR)

# Throw out BT1030 (not ST), CPDM0785 (not ST), MUV006 (ST YAP1), Peds4 (low counts), and BT165PDX (low counts)
samples_of_interest <- c('MUV043','MUV043Nuc1','MUV043Nuc2','MUV056')
st <- merge(st, y=pdx) %>% JoinLayers
st <- subset(st, sample %in% samples_of_interest)
st_list <- SplitObject(st, split.by = "sample")

st_list

In [ ]:
# Define QC thresholds
ncount_lower_threshold = 1000
ncount_upper_threshold = 1000000
nfeature_lower_threshold = 1000
nfeature_upper_threshold = 10000

# Plot threshods
options(repr.plot.width = 16, repr.plot.height =6)
VlnPlot(st, features="nCount_RNA", group.by='sample', log=TRUE) + 
    geom_hline(yintercept=ncount_lower_threshold, color='red') + 
    geom_hline(yintercept=ncount_upper_threshold, color='red')
VlnPlot(st, features="nFeature_RNA", group.by='sample') +
    geom_hline(yintercept=nfeature_lower_threshold,color='red') + 
    geom_hline(yintercept=nfeature_upper_threshold,color='red')


In [ ]:
l1cam_analysis <- function(seuratobj){
    seuratobj <- seuratobj %>% 
    subset(nFeature_RNA > nfeature_lower_threshold & nFeature_RNA < nfeature_upper_threshold &
                        nCount_RNA > ncount_lower_threshold & nCount_RNA < ncount_upper_threshold) %>%
    SCTransform(verbose = FALSE, vst.flavor="v2") %>%
    subset(malignant == "Malignant")

    l1cam_expr <- seuratobj[["SCT"]]@data["L1CAM", ]

    # Calculate the thresholds
    low_threshold <- quantile(l1cam_expr, 0.25)    # 25th percentile
    high_threshold <- quantile(l1cam_expr, 0.75)   # 75th percentile
    
    # Create a new metadata column with "high," "middle," or "low"
    seuratobj$l1cam_exp <- cut(
      l1cam_expr,
      breaks = c(-Inf, low_threshold, high_threshold, Inf),
      labels = c("low", "middle", "high"),
      right = FALSE
    )
    Idents(seuratobj) <- seuratobj$l1cam_exp #%>%
    #FindMarkers(cells.1 = "high", cells.2 = "low")
    return(seuratobj)
}

st_list <- lapply(X = st_list, FUN = l1cam_analysis) # loops over each element in list (donor)

In [ ]:
differential_exp <- function(seuratobj,qval_threshold=0.2){
    markers <- FindMarkers(seuratobj,ident.1="high",ident.2="low") %>%
    subset(p_val_adj < qval_threshold)
    sample <- seuratobj$sample %>% unique
    results[sample] <- markers
}
lapply(X = st_list, FUN = differential_exp)

# INTS1 and TTYH1 are differentially expressed in 2 or more samples.


# Dead code

In [ ]:
# merge into 1 dataset
gojo <- merge(pf, y=c(sp,st,pdx,pairs)) %>% JoinLayers
gojo
# list samples
get_sample_names <- function(seuratobject){
    return(seuratobject@meta.data$sample %>% unique())
}
get_sample_names(gojo)

In [ ]:
# https://statomics.github.io/singleCellCourse/lab4_CuomoTemplate.html#15_Trajectory_inference
g_alt_list <- SplitObject(gojo, split.by = "sample")
# get only the ZFTA samples
zfta_samples <- c('BT165PDX','MUV043','MUV043Nuc1','MUV043Nuc2','MUV056','Peds4')

g_alt_list <- g_alt_list[zfta_samples]
g_alt_list <- lapply(X = g_alt_list, FUN = function(x) { # loops over each element in list (donor)
    x <- SCTransform(x, verbose = TRUE, vst.flavor="v2")
    x <- RunPCA(x, verbose = FALSE, reduction.name="normalized.pca")
})
features <- SelectIntegrationFeatures(object.list = g_alt_list, nfeatures = 3000)
g_alt_list <- PrepSCTIntegration(object.list = g_alt_list, anchor.features = features)
anchors <- FindIntegrationAnchors(object.list = g_alt_list, normalization.method = "SCT",
    anchor.features = features)
# Extract features from each SeuratObject
feature_lists <- lapply(g_alt_list, function(obj) rownames(obj))
# Find the intersection of features across all SeuratObjects
common_features <- Reduce(intersect, feature_lists)
g_alt <- IntegrateData(anchorset = anchors, features.to.integrate=common_features, normalization.method = "SCT",k.weight=30)

In [ ]:
g_alt_list

In [ ]:
VlnPlot(g_alt, features = 'L1CAM', assay='integrated', group.by='annotation',layer='scale.data')

In [ ]:
correlations <- list()

# Extract the expression data
expr_data <- g_alt[["integrated"]]@scale.data

# Get the expression of L1CAM
l1cam_expr <- expr_data["L1CAM", ]

# Loop through each feature
for (feature in rownames(g_alt)) {
  # Get the expression of the current feature
  if (feature == 'L1CAM'){
      next
  }
  feature_expr <- expr_data[feature, ]
  
  # Calculate the correlation
  cor_value <- cor(l1cam_expr, feature_expr, use = "pairwise.complete.obs")
  
  # Store the correlation value
  correlations[[feature]] <- cor_value
}

# Convert to a data frame for better visualization
correlations_df <- data.frame(feature = names(correlations), correlation = unlist(correlations))

# Sort the correlations dataframe using dplyr
correlations_df <- correlations_df %>%
  arrange(desc(correlation))


In [ ]:
options(repr.plot.width = 16, repr.plot.height =32)

n <- 10 
# Get the top n features based on correlation
top_features <- head(correlations_df$feature, n)

# Initialize an empty list to store plots
plots <- list()

# Loop through each of the top features and create a FeatureScatter plot
for (feature in top_features) {
  cor_test <- cor.test(l1cam_expr,
                       expr_data[feature, ],
                       method = "pearson")
  
  # Extract the p-value
  p_value <- cor_test$p.value
  
  # Create a formatted string for the p-value
  p_value_text <- paste("p =", format(p_value, digits = 2))
  
  p <- FeatureScatter(g_alt, feature1="L1CAM", feature2=feature, slot="scale.data") +
       ggtitle(paste("Correlation with", feature)) +
       annotate("text", x = Inf, y = Inf, label = p_value_text, hjust = 1.1, vjust = 1.1, size = 4, color = "black")
  plots[[feature]] <- p
}

# Combine all plots using patchwork
combined_plot <- wrap_plots(plots) + plot_layout(ncol = 2)  # Adjust ncol as needed

# Display the combined plot
print(head(correlations_df,n))
print(combined_plot)

In [ ]:
# Assuming your list of SeuratObjects is called seurat_list
# Initialize a list to store correlation results

correlations_2 <- function(seurat_list){
    correlation_results <- list()
    # Loop through each SeuratObject
    for (i in seq_along(seurat_list)) {
      obj <- seurat_list[[i]]

      # Extract expression data
      expr_data <- obj[["SCT"]]@data

      # Get the expression of L1CAM
      l1cam_expr <- expr_data["L1CAM", ]
      
      # Initialize a vector to store correlations for this object
      cor_values <- numeric(nrow(expr_data))
      names(cor_values) <- rownames(expr_data)
      
      # Calculate correlations for each feature
      for (feature in rownames(expr_data)) {
        if (feature == 'L1CAM'){
          next
        }
        cor_values[feature] <- cor(l1cam_expr, expr_data[feature, ], use = "pairwise.complete.obs") %>% SuppressWarnings
      }
      
      # Store the results in the list
      correlation_results[[i]] <- cor_values
    }
    
    # Combine results into a single data frame
    combined_correlations <- do.call(cbind, correlation_results)
    return(combined_correlations)
    
    # Calculate the mean correlation across all samples
    mean_correlations <- rowMeans(combined_correlations, na.rm = TRUE)
    
    # Create a data frame with features and their mean correlations
    correlations_df <- data.frame(feature = names(mean_correlations), correlation = mean_correlations)
    
    # Sort by correlation
    correlations_df_sorted <- correlations_df[order(correlations_df$correlation, decreasing = TRUE), ]
    
    # Print the sorted correlations
    return(correlations_df_sorted)
}

In [ ]:
asdf <- correlations_2(g_alt_list)


In [ ]:
rownames(g_alt[["SCT"]]@data)

In [ ]:
mean_correlations <- rowMeans(asdf, na.rm = TRUE)
mean_correlations <- mean_correlations[order(-unlist(mean_correlations))]
head(mean_correlations,n=10)

In [ ]:
# scTransform should be performed per sample. See
# https://github.com/satijalab/seurat/issues/5306
# https://satijalab.org/seurat/archive/v4.3/sctransform_v2_vignette
gojo[["RNA"]] <- split(gojo[["RNA"]], f = gojo$sample)
DefaultAssay(gojo) <- "RNA"
gojo

In [ ]:
# Normalization
# For details see https://satijalab.org/seurat/articles/sctransform_vignette.html
# This takes a long time (>1h on 12 cores)
gojo <- SCTransform(gojo, verbose = TRUE, vst.flavor = "v2")

In [ ]:
# Checkpoint: normalization took forever so we save this as an .rds file
rds <- file.path('data','gojo_sctransformed_seuratobj.rds')
saveRDS(gojo, file = rds)

In [ ]:
# Checkpoint: load this file if you don't want to wait an hour for SCTransform.
rds <- file.path('data','gojo_sctransformed_seuratobj.rds')
gojo <- readRDS(file = rds)
gojo

In [ ]:
# If we cluster and plot after normalization but before batch correction, we largely get a soup that doesn't segregate by
# sample or by annotation.
gojo <- RunPCA(gojo, verbose = FALSE, reduction.name="normalized.pca")
gojo <- RunUMAP(gojo, dims = 1:30, verbose = FALSE, reduction="normalized.pca",reduction.name="normalized.umap")
gojo <- FindNeighbors(gojo, dims = 1:30, verbose = FALSE, reduction="normalized.pca",graph.name="normalized.snn")
gojo <- FindClusters(gojo, verbose = FALSE, graph.name="normalized.snn", cluster.name="normalized.clusters")


In [ ]:
print(c('Cluster similarity to sample IDs: ',mclust::adjustedRandIndex(
    gojo[[]]$sample,
    gojo[[]]$normalized.clusters)))
print(c('Cluster similarity to cell types: ',mclust::adjustedRandIndex(
    gojo[[]]$annotation,
    gojo[[]]$normalized.clusters)))

options(repr.plot.width = 16, repr.plot.height =8)
DimPlot(gojo, reduction="normalized.umap", label=TRUE, group.by="annotation", label.size=6, repel=TRUE)

In [ ]:
# Batch correction
# TODO: wrap in function
gojo <- gojo %>% IntegrateLayers(
    method = HarmonyIntegration,
    orig.reduction = "normalized.pca", new.reduction = "harmony",
    normalization.method = "SCT"
)
gojo <- RunUMAP(gojo, dims = 1:30, verbose = FALSE, reduction="harmony",reduction.name="harmony.umap")
gojo <- FindNeighbors(gojo, dims = 1:30, verbose = FALSE, reduction="harmony",graph.name="harmony.snn")
gojo <- FindClusters(gojo, verbose = FALSE, graph.name="harmony.snn", cluster.name="harmony.clusters")

print(c('Cluster similarity to sample IDs: ',mclust::adjustedRandIndex(
    gojo[[]]$sample,
    gojo[[]]$harmony.clusters)))
print(c('Cluster similarity to cell types: ',mclust::adjustedRandIndex(
    gojo[[]]$annotation,
    gojo[[]]$harmony.clusters)))


In [ ]:
options(repr.plot.width = 16, repr.plot.height =8)
DimPlot(gojo, reduction="harmony.umap", label=TRUE, group.by="annotation", label.size=6, repel=TRUE)
DimPlot(gojo, reduction="harmony.umap", label=TRUE, group.by="harmony.clusters", label.size=6, repel=TRUE)

In [ ]:
VlnPlot(gojo, features = 'L1CAM', group.by='annotation')
# L1CAM restricted to PF-Neuronal-Precursor-like and ST cell types.
VlnPlot(gojo, features = 'L1CAM', group.by='sample')
# BT1030 and CPDM0785 do not resemble RELA ependymomas in L1CAM expression.
# MUV006 (ST-YAP1) does not express L1CAM.

In [ ]:
microglia_markers = c('CD14', 'FCER1G', 'CSF1R')
tcell_markers = c('CD3E', 'CD4', 'CD8A')
opc_markers = c('OLIG1', 'APOD', 'PDGFRA')
oligodendrocyte_markers = c('MBP', 'PLP1', 'MOG')
other_markers = c('L1CAM')
markers = c(microglia_markers,tcell_markers,opc_markers,oligodendrocyte_markers,other_markers)
DotPlot(gojo, features = markers, group.by = "annotation") #+ RotatedAxis()

In [ ]:
#TODO: genes correlated with L1CAM expression
FeaturePlot(gojo, features = "L1CAM",pt.size=2,reduction='harmony.umap')